# Multiple terms of the same class

In [1]:
import oommfc as oc
import discretisedfield as df
import micromagneticmodel as mm

p1 = (-10e-9, 0, 0)
p2 = (10e-9, 1e-9, 1e-9)
cell = (1e-9, 1e-9, 1e-9)
region = df.Region(p1=p1, p2=p2)
mesh = df.Mesh(region=region, cell=cell)

The mesh is

In [2]:
mesh.k3d()

Output()

The system has a Hamiltonian, which consists of only uniaxial anisotropy energy term.

In [3]:
K1 = 1e5  # uniaxial anisotropy constant (J/m**3)
u1 = (0, 0, 1)  # uniaxial anisotropy axis
K2 = 2e5  # uniaxial anisotropy constant (J/m**3)
u2 = (0, 1, 0)  # uniaxial anisotropy axis
system = mm.System(name='multiple_terms')
#system.energy = #mm.UniaxialAnisotropy(K=K1, u=u1, name='ua1') + mm.UniaxialAnisotropy(K=K2, u=u2, name='ua2')
system.energy = mm.Zeeman(H=(0, 0, 1e5)) + mm.Zeeman(H=(0, 1e5, 0), name='z2')

We are going to minimise the system's energy using `oommfc.MinDriver` later. Therefore, we do not have to define the system's dynamics equation. Finally, we need to define the system's magnetisation (`system.m`). We are going to make it random with $M_\text{s}=8\times10^{5} \,\text{Am}^{-1}$

In [4]:
import random
import discretisedfield as df

Ms = 8e5  # saturation magnetisation (A/m)

def m_fun(pos):
    return [2*random.random()-1 for i in range(3)]

system.m = df.Field(mesh, dim=3, value=m_fun, norm=Ms)

The magnetisation, we set is

In [5]:
system.m.k3d_vector(color_field=system.m.z)

Output()

Now, we can minimise the system's energy by using `oommfc.MinDriver`.

In [6]:
md = oc.MinDriver()
md.drive(system)

Running OOMMF (ExeOOMMFRunner) [2020/06/30 11:50]... (2.1 s)


We expect that now all magnetic moments are aligned parallel or antiparallel to the anisotropy axis (in the $z$-direction).

In [7]:
system.m.k3d_vector(color_field=system.m.z)

Output()

In [8]:
system.table.data

,max_mxHxm,E,delta_E,bracket_count,line_min_count,conjugate_cycle_count,cycle_count,cycle_sub_count,energy_calc_count,E_zeeman,E_z2,iteration,stage_iteration,stage,mx,my,mz
0,0.067487,-2.843445e-21,-4.201687e-34,27.0,2.0,21.0,23.0,2.0,30.0,-1.421723e-21,-1.421723e-21,28.0,28.0,0.0,5.090218e-08,0.707107,0.707107
